In [26]:
import ocloc
import importlib
importlib.reload(ocloc)

<module 'ocloc' from '/Users/localadmin/Dropbox/GitHub/ocloc/src/ocloc/ocloc.py'>

In [27]:
import sys
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from statistics import mean
# Importing the main code.
module_path = os.path.abspath(os.path.join('../src/ocloc'))
if module_path not in sys.path:
    sys.path.append(module_path)
from ocloc import ProcessingParameters, ClockDrift, suppress_stdout, confidence_interval_95

First, we load the clock drift object we used for getting stable results.

In [138]:
file_name = "/Users/localadmin/Dropbox/GitHub/ocloc/tutorials/bootstrap/clockdrift.obj"
path_2_clockdrift = os.path.join(file_name)
with open(path_2_clockdrift, 'rb') as f:
    cd = pickle.load(f)
# To plot the correlations uncomment the following:
#for sta in cd.stations:
#    if sta.needs_correction:
#        cd.plot_correlation_beforeNafter_correction(sta.code, "KEF")

Then, we load the pkl where we saved all the bootstrap iterations

In [11]:
output_dir = "/Users/localadmin/Dropbox/GitHub/ocloc/tutorials/bootstrap/"
with open(output_dir + "refinedInversion_bootstrap.pkl", 'rb') as f:
    results = pickle.load(f)

In [8]:
snr_trh = 30
dist_trh = 2.5

In [156]:
iteration = "snr_trh_" + str(snr_trh) + "__dist_trh_" + str(dist_trh)
df = pd.DataFrame(columns=['Station', 'Parameter', 'mean',
                           'conf_inter', 'std'])

for i, inverse_method in enumerate([
        "weighted a ",
        "weighted b ",
    ]):
    for station in cd.stations:

        if not station.needs_correction:
            continue

        sta = station.code
        title = inverse_method + sta + iteration
        values = results[title]
        if len(values) == 0:
            print(sta)
            continue
        if inverse_method == "weighted a ":
            param = "Clock drift-a-[s/year]"
            values = values * 365
        else:
            param = "b value [s]"
        
        temp_df = pd.DataFrame(data=values, columns=[title])
        conf_inter = confidence_interval_95(values)
        conf_inter = [float(format(ci,".4f")) for ci in conf_inter]
        std = float(temp_df.std())
        mean = float(temp_df.mean())
        
        new_row = {'Station':sta, 
                   'Parameter': param,
                   'mean':str(round(mean, 5)),
                   'conf_inter':str(conf_inter),
                    'std':str(round(std, 5))}
        
        #append row to the dataframe
        df = df.append(new_row, ignore_index=True)
df.style.set_caption("The b value is calculated with respect to " +
                         cd.reference_time.strftime(format="%Y_%m_%d"))
display(df)
df.to_csv("/Users/localadmin/Downloads/clock_drift.csv")
#display(df.to_latex(index=False))

O23
O23


,Station,Parameter,mean,conf_inter,std
0,O01,Clock drift-a-[s/year],-0.00202,"[-0.0022, -0.0018]",0.00012
1,O02,Clock drift-a-[s/year],-0.0029,"[-0.0033, -0.0025]",0.00027
2,O03,Clock drift-a-[s/year],-0.00132,"[-0.0025, -0.0004]",0.00065
3,O04,Clock drift-a-[s/year],-0.00218,"[-0.0029, -0.0015]",0.00046
4,O06,Clock drift-a-[s/year],-0.00047,"[-0.0012, 0.0002]",0.00042
5,O08,Clock drift-a-[s/year],-0.00032,"[-0.0014, 0.0011]",0.0008
6,O10,Clock drift-a-[s/year],-0.00312,"[-0.0034, -0.0028]",0.0002
7,O11,Clock drift-a-[s/year],-0.00205,"[-0.0028, -0.0012]",0.00049
8,O14,Clock drift-a-[s/year],-0.00092,"[-0.0017, -0.0004]",0.00038
9,O15,Clock drift-a-[s/year],-0.17765,"[-0.0054, -0.0023]",5.27455


In [157]:
import pandas as pd

dates = ['2014-08-22', '2014-09-01', '2014-10-01', '2014-11-01',
         '2014-12-01', '2015-01-01', '2015-02-01', '2015-03-01',
         '2015-04-01', '2015-05-01', '2015-06-01', '2015-07-01',
         '2015-08-30']


In [158]:
import obspy
skew_values_file = "skew_values.csv"
path2file_skew = os.path.join(module_path, skew_values_file)
skew_df = pd.read_csv(path2file_skew, delimiter=",", header=0)

sensor_codes = skew_df["Sensor code"]
starttimes = skew_df["Start time"]
endtimes = skew_df["End time"]
ref_time = cd.reference_time

final_results = pd.DataFrame(columns = ["Sensor code", 
                                        "Date",
                                        "Timing error [s]"
                                       ])
for station in sensor_codes:
    for starttime in dates:
        if station in cd.station_names:
            sta_obj = cd.get_station(station)
            a = float(cd.solution["a ("+ station + ")"])
            b = float(cd.solution["b ("+ station + ")"])
            dt = (obspy.UTCDateTime(starttime) - ref_time)/86400
            dt_ins = a*(dt) + b

            new_row = {'Sensor code':station, 
                       'Date': obspy.UTCDateTime(starttime),
                       'Timing error [s]':str(dt_ins)[:5]}

            #append row to the dataframe
            final_results = final_results.append(new_row, ignore_index=True)
display(final_results)
final_results.to_csv("/Users/localadmin/Downloads/timing_error.csv")

,Sensor code,Date,Timing error [s]
0,O01,2014-08-22T00:00:00.000000Z,-0.31
1,O01,2014-09-01T00:00:00.000000Z,-0.33
2,O01,2014-10-01T00:00:00.000000Z,-0.39
3,O01,2014-11-01T00:00:00.000000Z,-0.45
4,O01,2014-12-01T00:00:00.000000Z,-0.51
...,...,...,...
216,O23,2015-04-01T00:00:00.000000Z,-0.49
217,O23,2015-05-01T00:00:00.000000Z,-0.51
218,O23,2015-06-01T00:00:00.000000Z,-0.54
219,O23,2015-07-01T00:00:00.000000Z,-0.56
